### 01 安装需要的依赖和部署预训练模型

In [ ]:
!python -m pip install paddlepaddle-gpu==2.4.1.post112 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!python -m pip install ppasr
!pip install pypinyin
!python -m pip install paddlespeech-ctcdecoders
!git clone https://github.com/AlexandaJerry/PPASR.git
%cd /content/PPASR
!python setup.py install

In [2]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1e7t3lOPj0PAgQHWVot0dRw8_NRoP27IN' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1e7t3lOPj0PAgQHWVot0dRw8_NRoP27IN" -O "PPASR.zip" && rm -rf /tmp/cookies.txt
!unzip /content/PPASR/PPASR.zip -d /content
!cp -RT /content/PPASR_V2-conformer_online-fbank-超大数据集/configs/ /content/PPASR/configs/
!cp -RT /content/PPASR_V2-conformer_online-fbank-超大数据集/dataset/ /content/PPASR/dataset/
!cp -RT /content/PPASR_V2-conformer_online-fbank-超大数据集/models/ /content/PPASR/models/
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1--Sy7fcquKBR8PNIVW-quDlSD1fCNcGk' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1--Sy7fcquKBR8PNIVW-quDlSD1fCNcGk" -O "zh_giga.no_cna_cmn.prune01244.klm" && rm -rf /tmp/cookies.txt
!cp -RT /content/PPASR/zh_giga.no_cna_cmn.prune01244.klm /content/PPASR/lm/
%cd /content/PPASR
!python export_model.py --resume_model=models/conformer_online_fbank/best_model/

--2022-12-21 02:06:28--  https://docs.google.com/uc?export=download&confirm=t&id=1e7t3lOPj0PAgQHWVot0dRw8_NRoP27IN
Resolving docs.google.com (docs.google.com)... 142.250.4.102, 142.250.4.101, 142.250.4.138, ...
Connecting to docs.google.com (docs.google.com)|142.250.4.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-8s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ad207mv95ra01051s697qc321o23muuc/1671588375000/03760087302420967133/*/1e7t3lOPj0PAgQHWVot0dRw8_NRoP27IN?e=download&uuid=41b7160d-08ab-41fc-8749-964db2b1580c [following]
--2022-12-21 02:06:28--  https://doc-14-8s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ad207mv95ra01051s697qc321o23muuc/1671588375000/03760087302420967133/*/1e7t3lOPj0PAgQHWVot0dRw8_NRoP27IN?e=download&uuid=41b7160d-08ab-41fc-8749-964db2b1580c
Resolving doc-14-8s-docs.googleusercontent.com (doc-14-8s-docs.googleusercontent.com)... 74.125.24.132, 2404:



---



---



### 02 导入谷歌云盘中的音频压缩文件

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
wav_file = 'data.zip'#@param {type:"string"}
wav_path = "/content/drive/MyDrive/" + wav_file
!unzip $wav_path -d /content/PPASR/raw_wav

Archive:  /content/drive/MyDrive/data.zip
  inflating: /content/PPASR/raw_wav/lj1216.wav  
  inflating: /content/PPASR/raw_wav/aam0511.wav  
  inflating: /content/PPASR/raw_wav/alv0805.wav  
  inflating: /content/PPASR/raw_wav/hp0317.wav  




---



---



### 03 长音频自动切片为短音频

In [7]:
import glob
import os
import shutil
import wave

import librosa
import soundfile

def length(src: str):
    if os.path.isfile(src) and src.endswith('.wav'):
        with wave.open(src, 'r') as w:
            return w.getnframes() / w.getframerate() / 3600
    elif os.path.isdir(src):
        total = 0
        for ch in [os.path.join(src, c) for c in os.listdir(src)]:
            total += length(ch)
        return total
    return 0


print('Environment initialized successfully.')

# Configuration for data paths
raw_path = '/content/PPASR/raw_wav'  # Path to your raw, unsliced recordings

########################################

assert os.path.exists(raw_path) and os.path.isdir(raw_path), 'The chosen path does not exist or is not a directory.'
print('Raw recording path:', raw_path)
print()
print('===== Recording List =====')
raw_filelist = glob.glob(f'{raw_path}/*.wav', recursive=True)
raw_length = length(raw_path)
if len(raw_filelist) > 5:
    print('\n'.join(raw_filelist[:5] + [f'... ({len(raw_filelist) - 5} more)']))
else:
    print('\n'.join(raw_filelist))
print()
print(f'Found {len(raw_filelist)} valid recordings with total length of {round(raw_length, 2)} hours.')

sliced_path = '/content/PPASR/sliced_wav'  # Path to hold the sliced segments of your recordings

# Slicer arguments
db_threshold_ = -40. #@param {type:"number"}
min_length_ = 10000 #@param {type:"number"}
win_l_ = 800 #@param {type:"number"}
win_s_ = 40 #@param {type:"number"}
max_silence_kept_ = 1000 #@param {type:"number"}

# Number of threads (based on your CPU kernels)
num_workers = 5

########################################

assert 'raw_path' in locals().keys(), 'Raw path of your recordings has not been specified.'
assert not os.path.exists(sliced_path) or os.path.isdir(sliced_path), 'The chosen path is not a directory.'
os.makedirs(sliced_path, exist_ok=True)
print('Sliced recording path:', sliced_path)

from slicer import Slicer
from concurrent.futures import ThreadPoolExecutor, wait, ALL_COMPLETED

def slice_one(in_audio):
    audio, sr = librosa.load(in_audio, sr=None)
    slicer = Slicer(
        sr=sr,
        db_threshold=db_threshold_,
        min_length=min_length_,
        win_l=win_l_,
        win_s=win_s_,
        max_silence_kept=max_silence_kept_
    )
    chunks = slicer.slice(audio)
    for i, chunk in enumerate(chunks):
        soundfile.write(os.path.join(sliced_path, f'%s_slice_%04d.wav' % (os.path.basename(in_audio).rsplit('.', maxsplit=1)[0], i)), chunk, sr)


print('Slicing your recordings may take several minutes. Please wait.')
thread_pool = ThreadPoolExecutor(max_workers=num_workers)
tasks = []
for file in raw_filelist:
    tasks.append(thread_pool.submit(slice_one, file))
wait(tasks, return_when=ALL_COMPLETED)
print()
print('===== Segment List =====')
sliced_filelist = glob.glob(f'{sliced_path}/*.wav', recursive=True)
sliced_length = length(sliced_path)
if len(sliced_filelist) > 5:
    print('\n'.join(sliced_filelist[:5] + [f'... ({len(sliced_filelist) - 5} more)']))
else:
    print('\n'.join(sliced_filelist))
print()
print(f'Sliced your recordings into {len(sliced_filelist)} segments with total length of {round(sliced_length, 2)} hours.')

Environment initialized successfully.
Raw recording path: /content/PPASR/raw_wav

===== Recording List =====
/content/PPASR/raw_wav/alv0805.wav
/content/PPASR/raw_wav/aam0511.wav
/content/PPASR/raw_wav/lj1216.wav
/content/PPASR/raw_wav/hp0317.wav

Found 4 valid recordings with total length of 0.37 hours.
Sliced recording path: /content/PPASR/sliced_wav
Slicing your recordings may take several minutes. Please wait.
executing 'slice' costed 28.049s
executing 'slice' costed 29.258s
executing 'slice' costed 35.529s
executing 'slice' costed 38.942s

===== Segment List =====
/content/PPASR/sliced_wav/aam0511_slice_0010.wav
/content/PPASR/sliced_wav/hp0317_slice_0005.wav
/content/PPASR/sliced_wav/alv0805_slice_0005.wav
/content/PPASR/sliced_wav/lj1216_slice_0007.wav
/content/PPASR/sliced_wav/alv0805_slice_0015.wav
... (75 more)

Sliced your recordings into 80 segments with total length of 0.3 hours.




---



---



### 04 切片后的短音频自动转写

In [9]:
%%shell
cd /content/PPASR
#ls ./sliced_wav/*.wav
for file in ./sliced_wav/*.wav;  #一级目录下的内容-->并不递归显示！ 
  do  
     python infer_path.py --wav_path=$file;  #${file}代表的是文件的全路径
  done 

初始化解码器...
language model: model path = lm/zh_giga.no_cna_cmn.prune01244.klm, is_character_based = True, max_order = 5, dict_size = 0
初始化解码器完成!
[2022-12-21 02:16:48.733064 INFO   ] inference_predictor:__init__:71 - 已加载模型：models/conformer_online_fbank/infer/
消耗时间：460ms, 识别结果: 朋友是律师爸爸喜欢看书哥哥是医生
初始化解码器...
language model: model path = lm/zh_giga.no_cna_cmn.prune01244.klm, is_character_based = True, max_order = 5, dict_size = 0
初始化解码器完成!
[2022-12-21 02:17:02.707639 INFO   ] inference_predictor:__init__:71 - 已加载模型：models/conformer_online_fbank/infer/
消耗时间：396ms, 识别结果: 我喜欢看电视他们是护士高鹏和
初始化解码器...
language model: model path = lm/zh_giga.no_cna_cmn.prune01244.klm, is_character_based = True, max_order = 5, dict_size = 0
初始化解码器完成!
[2022-12-21 02:17:15.158068 INFO   ] inference_predictor:__init__:71 - 已加载模型：models/conformer_online_fbank/infer/
消耗时间：383ms, 识别结果: 徐涛是邻居他不叫高峰
初始化解码器...
language model: model path = lm/zh_giga.no_cna_cmn.prune01244.klm, is_character_based = True, max_order = 5, dict_size = 0

In [10]:
import os
import sys
import numpy as np
from pypinyin import pinyin, lazy_pinyin, Style
import re

root_dir = "/content/PPASR/sliced_wav"
pattern = re.compile(r'(.*)\.txt$')
r="[_.!+-=——,$%^,。?、~@#￥%……&*《》<>「」{}【】()（）/''\n ]"

for root, dir, files in os.walk(root_dir):
	for filename in files:
		#print(filename)
		output = pattern.match(filename)
		if output is not None:
			print(root, filename)
			text_file = open(root+"/"+filename)
			line = text_file.read().strip()
			line = line.replace("，", "")
			line = re.sub(r,"",line)
			pinyin =  lazy_pinyin(line, style=Style.TONE3, errors='default', strict=False, v_to_u=False, neutral_tone_with_five=True,tone_sandhi=True)
			pinyinline = ' '.join(pinyin)
			print(line)
			target_text_file = open(root+"/"+output.group(1)+".txt", "w")
			target_text_file.write(pinyinline)
			target_text_file.close()

/content/PPASR/sliced_wav aam0511_slice_0002.txt
徐涛是邻居他不叫高峰
/content/PPASR/sliced_wav alv0805_slice_0000.txt
朋友是李氏爸爸喜欢看书格格是一生
/content/PPASR/sliced_wav aam0511_slice_0013.txt
我叫木有房高峰是律师
/content/PPASR/sliced_wav alv0805_slice_0004.txt
我七点一刻去见李石朋友不是我们是邻居
/content/PPASR/sliced_wav aam0511_slice_0028.txt
北京比我说的是这样的六个小时
/content/PPASR/sliced_wav alv0805_slice_0017.txt
二十五现在六点半我们去听音乐会吧好吗
/content/PPASR/sliced_wav lj1216_slice_0013.txt
奥斯路现在五点半去音乐会吧好吗
/content/PPASR/sliced_wav hp0317_slice_0004.txt
八月八号是星期五高鹏和许涛是邻居
/content/PPASR/sliced_wav alv0805_slice_0015.txt
高鹏的女儿喜欢跳舞她穿白色的衣服哥哥喜欢看书
/content/PPASR/sliced_wav alv0805_slice_0001.txt
我喜欢看电视他们是护士
/content/PPASR/sliced_wav aam0511_slice_0024.txt
看电视他看
/content/PPASR/sliced_wav hp0317_slice_0006.txt
我有一个女儿她不叫高朋朋友不是我们是邻居看书你觉得有意思吗
/content/PPASR/sliced_wav hp0317_slice_0001.txt
他们是护士高鹏和许涛是邻居他不叫高鹏
/content/PPASR/sliced_wav aam0511_slice_0011.txt
弟弟喜欢跳舞许涛没有女儿爸爸喜欢看书哥哥是一生
/content/PPASR/sliced_wav lj1216_slice_0000.txt
朋友是律师爸爸喜欢看书哥哥是医生我喜欢看电视
/content

In [ ]:
#from IPython.display import Audio
#wn = Audio('/content/PPASR/sliced_wav/aam0511_slice_0006.wav', autoplay=True)
#with open('/content/PPASR/sliced_wav/aam0511_slice_0006.txt', 'r') as f:
    #print(f.read())
#display(wn)

### 05 保存短音频和对应转写到谷歌云盘

In [14]:
import os, tarfile
import os
from google.colab import files
save_as = 'mfa_prepare.tar'#@param {type:"string"}
def make_targz_one_by_one(output_filename, source_dir):
  tar = tarfile.open(output_filename,"w")
  for root,dir_name,files_list in os.walk(source_dir):
    for file in files_list:
      pathfile = os.path.join(root, file)
      tar.add(pathfile)
  tar.close()
 
  # files.download(output_filename)

make_targz_one_by_one(save_as, '/content/PPASR/sliced_wav')
savename = "/content/PPASR/" + save_as
!cp -r $savename /content/drive/MyDrive